In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# pd.read_excel?
df=pd.read_excel('HW cluster data.xlsx',sheet_name=0,
              header=0,
              index_col=0,nrows=8
              )
df#groupby('Clus')

,X,Y,Cluster
case #,,,
1,2,1,1
2,3,2,1
3,4,0,1
4,6,2,2
5,10,4,2
6,4,5,3
7,5,6,3
8,6,4,3


In [ ]:
df.loc[:,['cluster_mean_'+str(i) for i in df.columns if i!='Cluster']]=df.groupby('Cluster').transform('mean').values
df

,X,Y,Cluster,cluster_mean_X,cluster_mean_Y
case #,,,,,
1,2,1,1,3.0,1.0
2,3,2,1,3.0,1.0
3,4,0,1,3.0,1.0
4,6,2,2,8.0,3.0
5,10,4,2,8.0,3.0
6,4,5,3,5.0,5.0
7,5,6,3,5.0,5.0
8,6,4,3,5.0,5.0


In [ ]:
# values of observation across variables
# mean of variables axis 1

# deviance of variable about mean then their squares
# Finally sum of squared deviances
var_cols=['X','Y']


def within_cluster_sumofSquares(d,var_cols):
  arr=d.loc[:,var_cols].values#shape=obs,var

  cluster_mean_vec=arr.mean(axis=0)#shape=var,
  #cab be directly vectorised/broadcasted


  deviance=arr-cluster_mean_vec
  square_deviance=pow(deviance,2)
  sum_square_deviance=square_deviance.sum().sum()#first summ across each variable for all observation and then sum of deviance across each variable
  # print(deviance)
  return sum_square_deviance

df_cluster_sumofSquares=df.groupby(['Cluster']).apply(within_cluster_sumofSquares,var_cols=var_cols)
df_cluster_sumofSquares



Cluster
1     4.0
2    10.0
3     4.0
dtype: float64

In [ ]:
wss=df_cluster_sumofSquares.sum()
wss

18.0

# BSS
bss is primarily associated with deviance of cluster mean vector from the global mean vector

The computation has to also consider the number of data points within each cluster


-- Compute local mean and global means


>   Create a l2 deviance vector for the distance between two means

>   Create count vector for each cluster


>> Compute product of two and then vector sum to generate BSS



In [ ]:
global_mean_vector=df.loc[:,var_cols].mean().values#shape var,

cluster_mean_vector=df.groupby('Cluster')[var_cols].mean().values# cluster,var
cluster_count=df.Cluster.value_counts(sort=False).values# cluster,

#lets compute the l2 diviance first so that the vector shape gets aligned

l2_deviance=np.sum(np.power(cluster_mean_vector-global_mean_vector,2),axis=1)#we need square deviance for each cluster computed across all the variables
l2_deviance

weighted_l2Deviance=l2_deviance*cluster_count
weighted_l2Deviance

array([24., 18., 12.])

In [ ]:
bss=sum(weighted_l2Deviance)
bss

54.0

# TSS
We will verify TSS= BSS +Wss

TSS is the deviance across global_mean
We can consider the entire dataset as the global cluster and computer square of deviance

In [ ]:
tss=within_cluster_sumofSquares(df,var_cols)
tss

72.0

# R2
R square determines how much variation is being explained by the new group of means as compared to the previous ones

Previous one in generalised case refers to the Global mean 

r2=1-(WSS/TSS)
>=BSS/TSS


In [ ]:
r2=bss/tss
r2

0.75

# If each data point was itself a cluster

In [ ]:
# If each data point was itself a cluster then the WSS would have been 0 and no variation is left to be explained when we have considered all the data points
#BSS would have been equal to TSS and r2 thus be 1
BSS=TSS=72
WSS=0

# IF we only had one cluster 

In [ ]:
# TSS=same as before as it is a holistic data property
# WSS=TSS as we only have one cluster
# BSS=0 as there is only one local mean that is the global mean 
#Thus r2 would have been 0 (BSS/TSS)

# DISSIMILARITY MATRIX

In [ ]:
arr=df.loc[:,var_cols].values
arr#shape = obs,vals
obs,vals=arr.shape
temp_1=arr.reshape(obs,1,vals)
temp_2=arr.reshape(1,obs,vals)

#for every pair of observation we will achive deviance if we computed temp_1-temp_2
dev_arr=temp_1-temp_2
#formulate this design and then apply any function along the given axis


In [ ]:
def deviance_square_sum(arr):
  return sum(pow(arr,2))
dis_arr=np.apply_along_axis(deviance_square_sum,arr=dev_arr,axis=2)
dis_arr

array([[ 0,  2,  5, 17, 73, 20, 34, 25],
       [ 2,  0,  5,  9, 53, 10, 20, 13],
       [ 5,  5,  0,  8, 52, 25, 37, 20],
       [17,  9,  8,  0, 20, 13, 17,  4],
       [73, 53, 52, 20,  0, 37, 29, 16],
       [20, 10, 25, 13, 37,  0,  2,  5],
       [34, 20, 37, 17, 29,  2,  0,  5],
       [25, 13, 20,  4, 16,  5,  5,  0]])

In [ ]:
# dis_chk=np.zeros(shape=(obs,obs))
# for i in range(len(arr)):
#   for j in range(len(arr)):
#     dis_chk[i,j]=sum(pow((arr[i]-arr[j]),2))
# dis_chk-dis_arr


In [ ]:
df.loc[:,var_cols].to_excel('data_file.xlsx')

# SINGLE, COMPLTE,Average,Centroid WARD Linkage


In [ ]:
#Single

#We ought to compute distance of the test point to the nearest point withing each cluster and then assign the new point to the respective cluster

#I intend to use group by and apply linkage functions



def linkage(d,pivot_pt,var_cols,link='single',verbose=0):
  def l2_norm(a,b):
    return pow((a-b),2).sum(axis=1)

  arr=d.loc[:,var_cols].values#shape=obs,var
  #apply broadcast
  dist=l2_norm(arr,pivot_pt)
  if link=='single':
    p_dist=np.min(dist)
  elif link=='complete':
    p_dist=np.max(dist)
  elif link=='average':
    p_dist=np.mean(dist)
  elif link=='centroid':
    centroid=np.mean(arr,axis=0).reshape(1,-1)
    p_dist=l2_norm(centroid,pivot_pt)[0]
  else:#WARD
    p_dist=np.sum(dist)
  if verbose:
    return (p_dist,dist)
  return p_dist


linkage(df,np.array([5,2]),var_cols,verbose=True)



(1, array([10,  4,  5,  1, 29, 10, 16,  5]))

In [ ]:
test_point=np.array([5,2])
df.groupby('Cluster').apply(linkage,test_point,var_cols,'single')#Cluster 2

Cluster
1    4
2    1
3    5
dtype: int64

In [ ]:
df.groupby('Cluster').apply(linkage,test_point,var_cols,'complete')#Cluster 1

Cluster
1    10
2    29
3    16
dtype: int64

In [ ]:
df.groupby('Cluster').apply(linkage,test_point,var_cols,'average')#Cluster 1

Cluster
1     6.333333
2    15.000000
3    10.333333
dtype: float64

In [ ]:
df.groupby('Cluster').apply(linkage,test_point,var_cols,'centroid')#Cluster 1

Cluster
1     5.0
2    10.0
3     9.0
dtype: float64

In [ ]:
df.groupby('Cluster').apply(linkage,test_point,var_cols,'ward')#Cluster 1

Cluster
1    19
2    30
3    31
dtype: int64

In [ ]:
#https://blogs.sas.com/content/iml/2013/03/27/compute-distance.html

pseudo F criterion --> Local max
Suggests that the number of clusters at this local max was better to explain the data than 1 more and 1 less cluster

Ans 21--> 17
22--> 27 customers
23--> 5
24--> 85
25--> .502

26-->.0497
27-->


4,2



. Masters degree, medium salary, medium concern about social status is CLUSTER 2

 High school diploma, low salary, unconcerned about social status is CLUSTER 1

 . Masters degree, high salary, low savings
 CLSUTER 4



In [ ]:
#